# This file is to finish the udacity nano degree problem for Anna_KaRNNa with Pytorch

This file is to build a chracter-wise RNN trained on Anna Kerenia.

## 1. Load Packages

In [8]:
import time
from collections import namedtuple
import numpy as np
import torch

## 2. Load Data

In [14]:
with open('anna.txt','r') as f:
    text = f.read()
vocab =sorted(set(text))
vocab_to_int = {c: i for i,c in enumerate(vocab)}
int_to_vocab =dict(enumerate(vocab))
encoded = np.array([vocab_to_int[c] for c in text])

One of the best first line of a book ever.

In [54]:
lr = 0.001

In [55]:
text[:100]

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

## 3. Make training mini-batches

In [26]:
# Follow the udacity's books function to create the batch-iterator here

In [126]:
def get_batches(arr, n_seqs, n_steps):
    '''Create a generator that returns mini-batches of size
       n_seqs x n_steps from arr.
    '''
    
    batch_size = n_seqs * n_steps
    n_batches = len(arr)//batch_size
    
    # Keep only enough characters to make full batches
    arr = arr[:n_batches * batch_size]
    # Reshape into n_seqs rows
    arr = arr.reshape((n_seqs, -1))
    
    for n in range(0, arr.shape[1], n_steps):
        # The features
        x = arr[:, n:n+n_steps]
        # The targets, shifted by one
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+n_steps]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y

In [29]:
# def get_batches(arr, batch_size, n_steps):
#     ''' Create generator that returns the batches of size 
#         batch_size * n_steps
#         Here we follow the udacity's notebook's rule, we will remove the last piece
#         of the samples smaller than the batch_size.
#     '''
#     size = batch_size * n_steps
#     n_batches = len(arr)/size
    
#     # Here we only keep enough numbers 
#     arr = arr[:size*n_batches]
    
#     # reshape batches into batch_size row
#     arr = arr.reshape((batch_size,-1))
    
#     for n in range(0,arr.shape[1],n_steps):
#         x = arr[:,n:n+n_steps]
#         y_temp = arr[:,n+1:n+n_steps+1]
        
#         # for the last batch, the y will be short with one.
#         # we will pad the last value with 0
        
#         y = np.zeros(x.shape)
#         y[:,:y_temp.shape[1]]=y_temp
        
#         yield x,y

In [143]:
n_seqs = 50
n_steps = 100
label_size = len(vocab)

In [144]:
print('x \n', x[:10, :10])
print('\n y \n', y[:10, :10])

('x \n', tensor([[[ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  1.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         ...,
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.]],

        [[ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  1.,  0.,  ...,  0.,  0.,  0.],
         ...,
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.]],

        [[ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         ...,
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  1.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.]],

        ...,

        [[ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  0., 

## 4. Model Building

In [145]:
# Here we build the LSTM neural network models.

In [146]:
## Need to rewrite this function

import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

class LSTM_Char(nn.Module):
    
    def __init__(self,hidden_dim,n_steps,n_layers,label_size,batch_size,drop_out=0.5,lr=0.001):
        
        super(LSTM_Char,self).__init__()
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.label_size = label_size
        self.n_layers = n_layers
        self.drop_out = drop_out
        self.dropout = nn.Dropout(drop_out)
        self.lstm = nn.LSTM(input_size= label_size ,hidden_size = hidden_dim,num_layers = n_layers,\
                            dropout=drop_out,batch_first=True)
        self.hidden = self.init_hidden(n_steps)
        self.out = nn.Linear(hidden_dim,label_size)
        self.init_weight()
        
    def init_hidden(self,n_seqs):
        ## Two layer of hidden, first is for the input x, second is for the hidden state
        weight = next(self.parameters()).data
        return (Variable(weight.new(self.n_layers,n_seqs,self.hidden_dim).zero_()),
                Variable(weight.new(self.n_layers,n_seqs,self.hidden_dim).zero_()))
    
    def forward(self,x,hc):
        # Here we put the hc in the tuple
        x,(h,c) = self.lstm(x,hc)
        x = self.dropout(x)
        
        # Put all the LSTM outputs together
        x = x.view(x.size()[0]*x.size()[1],self.hidden_dim)
        
        return x,(h,c)
    
    def predict(self,char,h=None,cuda=False,top_k=None):
        ''' Given a character, predict the next character.
            Return the predicted character and the hidden state.
        '''
        # Currently we don't consider cuda here.
        if h is None:
            h=self.init_hidden(1)
        x = np.array(vocab_to_int[char])
        x = one_hot_encode(x,self.label_size)
        
        inputs = Variable(torch.from_numpy(x),volatile = True)
        
        h = tuple(Variable(each.data,volatile = True) for each in h)
        
        out,h= self.forward(inputs,h)
        
        p = F.softmax(out).data
        
        if top_k is None:
            top_ch = np.arange(self.label_size)
        else:
            p,top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        p = p.numpy().squeeze()
        
        char = np.random.choice(top_ch, p=p/p.sum())
        
        return int_to_vocab[char],h
    
    def init_weight(self):
        initrange = 0.1
        #set bias tensor to all zeros
        self.out.bias.data.fill_(0)
        # set weight to randeom uniform
        self.out.weight.data.uniform_(-1,1)
        

In [147]:
def one_hot_encode(arr, n_labels):
    
    # Initialize the the encoded array
    one_hot = np.zeros((np.multiply(*arr.shape), n_labels), dtype=np.float32)
    
    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((arr.shape[0],arr.shape[1], n_labels))
    
    return one_hot

## 5. Model Training

In [152]:
net = LSTM_Char(hidden_dim = 512,n_steps=100,\
                n_layers = 2,label_size = len(vocab),\
                batch_size=128)

In [153]:
net.train()
opt = torch.optim.Adam(net.parameters(),lr=lr)
criterion = nn.CrossEntropyLoss()

In [154]:
# Here we can do the split between training and test data
val_frac = 0.2
val_index = int(len(encoded)*(1-val_frac))
data,val_data = encoded[:val_index],encoded[val_index:]

## 6. Check Results

In [155]:
epochs = 10
clip = 5
counter = 0
print_every = 10
for e in range(epoch):
    h = net.init_hidden(n_seqs)
    for x, y in get_batches(data,n_seqs,n_steps):
        counter += 1
        
        # Need to do one-hot encoding and make them the torch tensors
        x = one_hot_encode(x,label_size)
        x, y =torch.from_numpy(x),torch.from_numpy(y)
        
        inputs,targets = Variable(x), Variable(y)
        
        # Create new variables for the hidden state, otherwise we'd backprop
        # thourgh the entire training history
        ## Need to double check which options works better, detach or create new variables
        
        h = tuple([Variable(each.data) for each in h])
        
        net.zero_grad()
        
        output, h = net.forward(inputs,h)
        
        loss = criterion(output,targets.view(n_seqs*n_steps))
        
        loss.backward()
        
        # clip_grad_norm helps prevent the exploding gradient
        # double check this
        # nn.utils.clip_grad_norm(net.parameters(),clip)
        
        opt.step()
        
        if counter % print_every == 0:
            # Get validation loss
            val_h = net.init_hidden(n_seqs)
            val_losses = []
            for x, y in get_batches(val_data,n_seqs,n_steps):
                x = one_hot_encode(x,label_size)
                x,y = torch.from_numpy(x),torch.from_numpy(y)
                
                val_h = tuple([Variable(each.data,volatile= True) for each in val_h ])
                
                inputs, targets = Variable(x,volatile=True),Variable(y,volatile=True)
                
                output,val_h = net.forward(inputs,val_h)
                
                val_loss = criterion(output,targets.view(n_seqs*n_steps))
                
                val_losses.append(val_loss.data[0])
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.data[0]),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))
            

/Users/edwin/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:32: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
/Users/edwin/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:44: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/Users/edwin/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:46: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/Users/edwin/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:52: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/Users/edwin/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:56: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


('Epoch: 1/10...', 'Step: 10...', 'Loss: 5.2833...', 'Val Loss: 5.2839')
('Epoch: 1/10...', 'Step: 10...', 'Loss: 5.2833...', 'Val Loss: 5.2630')
('Epoch: 1/10...', 'Step: 10...', 'Loss: 5.2833...', 'Val Loss: 5.2471')
('Epoch: 1/10...', 'Step: 10...', 'Loss: 5.2833...', 'Val Loss: 5.2473')
('Epoch: 1/10...', 'Step: 10...', 'Loss: 5.2833...', 'Val Loss: 5.2451')
('Epoch: 1/10...', 'Step: 10...', 'Loss: 5.2833...', 'Val Loss: 5.2431')
('Epoch: 1/10...', 'Step: 10...', 'Loss: 5.2833...', 'Val Loss: 5.2421')
('Epoch: 1/10...', 'Step: 10...', 'Loss: 5.2833...', 'Val Loss: 5.2437')
('Epoch: 1/10...', 'Step: 10...', 'Loss: 5.2833...', 'Val Loss: 5.2445')
('Epoch: 1/10...', 'Step: 10...', 'Loss: 5.2833...', 'Val Loss: 5.2463')
('Epoch: 1/10...', 'Step: 10...', 'Loss: 5.2833...', 'Val Loss: 5.2429')
('Epoch: 1/10...', 'Step: 10...', 'Loss: 5.2833...', 'Val Loss: 5.2413')
('Epoch: 1/10...', 'Step: 10...', 'Loss: 5.2833...', 'Val Loss: 5.2418')
('Epoch: 1/10...', 'Step: 10...', 'Loss: 5.2833...'

KeyboardInterrupt: 